In [142]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [143]:
def _parse_line(line):

    rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
    }   

    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()
# url="http://192.168.20.58:8086"
url = "http://localhost:8086"

In [165]:
start = 1696343921
# # end = 1705735851
# start = 1707740868
end = 1708065415

node = "caterham"

In [166]:
def experiment(start_time, end_time, filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = expTime.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment(start, end, '../../Datasets/experiments_raw.csv')
experiments.to_csv('../../Datasets/experiments_filtered.csv')
experiments.tail(200)

/tmp/ipykernel_80161/3383679728.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
/tmp/ipykernel_80161/3383679728.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
329,712,2024-01-18 23:49:33.9823415,2024-01-19 00:19:33.9854755,unl,random28,20,18,23,8,4,16,0.50,1.00,1705621773,1705623573
330,713,2024-01-19 00:23:08.5330606,2024-01-19 00:53:08.5417381,unl,random28,20,18,23,8,4,16,0.50,1.00,1705623788,1705625588
331,714,2024-01-19 00:56:42.9001595,2024-01-19 01:26:42.9031478,unl,random29,18,20,8,4,6,4,0.25,3.00,1705625802,1705627602
332,715,2024-01-19 01:30:16.3550833,2024-01-19 02:00:16.3582969,unl,random29,18,20,8,4,6,4,0.25,3.00,1705627816,1705629616
333,716,2024-01-19 02:03:50.1184798,2024-01-19 02:33:50.1214603,unl,random29,18,20,8,4,6,4,0.25,3.00,1705629830,1705631630
334,717,2024-01-19 02:37:23.5929531,2024-01-19 03:07:23.5977598,unl,random30,15,4,20,8,2,8,1.00,1.00,1705631843,1705633643
335,718,2024-01-19 03:10:57.9766699,2024-01-19 03:40:57.9794300,unl,random30,15,4,20,8,2,8,1.00,1.00,1705633857,1705635657
336,719,2024-01-19 03:44:32.5404468,2024-01-19 04:14:32.5472689,unl,random30,15,4,20,8,2,8,1.00,1.00,1705635872,1705637672
337,720,2024-01-19 04:18:07.9259423,2024-01-19 04:48:07.9860640,unl,random31,8,6,18,4,4,4,0.25,0.50,1705637887,1705639687
338,721,2024-01-19 04:51:40.8244720,2024-01-19 05:21:40.8270549,unl,random31,8,6,18,4,4,4,0.25,0.50,1705639900,1705641700


In [167]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [168]:
def from_influx_count(url, token, org, start_time, end_time,grouping_key):
    # start_time = 1693222601 
    # end_time = 1693228430    
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)
        
        # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()
        
    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                       ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                        '|> filter(fn: (r) => r._measurement == "deliverMessage") '
                        '|> group(columns: ["_measurement", "_field"], mode: "by") '
                        '|> count()')
    client.close()

    if data_frame.empty:
        count = 0
    else:
        count = data_frame["_value"].min().astype(int)
    return count

In [ ]:
#Validate data
validate = pd.DataFrame()

for index, row in experiments.iterrows():
    count = from_influx_count(url, token, org, row["startUnix"], row["endUnix"],"_measurement")
    d = {'experiment': [row['experiment']], 'count': [count], 'topology': [row["topology"]], 'd': [row["d"]],'dhi': [row["dhi"]],'dlo': [row["dlo"]],'dlazy': [row["dlazy"]],
        'dscore': [row["dscore"]],'dout': [row["dout"]],'gossipFactor': [row["gossipFactor"]],'interval': [row["interval"]],}
    aux = pd.DataFrame(data=d)
    validate = pd.concat([validate, aux])

# validate.head(200)

exps = experiments.merge(validate, on=['experiment', 'topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
exps = exps.loc[exps["count"]>0]

exps.head(10)

In [ ]:
#Remove fault executions

#get median
median = exps.drop(columns=['start','end','startUnix','endUnix', 'parameter', 'experiment']).drop_duplicates()
median = median.groupby(['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval']).agg({'count':['median', 'std', 'mean']})
median.columns = median.columns.droplevel(0)
median.reset_index(inplace=True)
median = median.loc[median['median'] > 0]

median.head(50)

In [ ]:
#Validate the data

validation = exps.merge(median, on=['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
validated = validation[(validation['count'] >= validation['mean']-(0.15*validation['std']))]
validated = validated.drop(columns=['startUnix', 'endUnix'])
validated.dropna()

validated.head(100)

In [ ]:
#Limit extraction to 15 minutes

experiments = validated.drop(columns=['count','median', 'mean','std'])#.rename(columns={'start':'originalStart','end':'originalEnd'})

# experiments['originalStart'] = pd.to_datetime(experiments["originalStart"], format='mixed')
# experiments['originalEnd'] = pd.to_datetime(experiments["originalEnd"], format='mixed')

# experiments['start'] = experiments['originalStart'] + pd.Timedelta(hours=0, minutes=7, seconds=30)
# experiments['end'] = experiments['originalEnd'] - pd.Timedelta(hours=0, minutes=7, seconds=30)

experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed").astype('int64') / 10**9
experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed").astype('int64') / 10**9

experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

experiments.to_csv('../../Datasets/experiments_valid.csv')
experiments.head(10)

In [ ]:
data = pd.read_csv('../../Datasets/experiments_valid.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.tail(10)

In [ ]:
def from_influx_message(url, token, org, start_time, end_time):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "message" and r.node == "'+node+'" and r._field == "type" and (r._value == 11 or r._value == 12 or r._value == 3 or r._value == 2)) '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')

    client.close()
    
    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', 'type']].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [ ]:
# message = pd.DataFrame()
execs = experiments#.drop(columns=['originalStart', 'originalEnd'])
# final=final.loc[final["identifier"] == 1]

for idx, exec in execs.iterrows():
    # print(exec["startUnix"], exec["endUnix"])
    query_mess = from_influx_message(url, token, org, exec["startUnix"], exec["endUnix"])

    query_mess["experiment"] = exec["experiment"]

    query_mess["start"] = exec["start"]
    query_mess["end"] = exec["end"]
            
    query_mess["_time"] = pd.to_datetime(query_mess["_time"])
        # message = pd.concat([message,query_mess])

    query_mess.to_csv('../../Datasets/partialTables/mess_'+node+'_'+str(exec["experiment"])+'.csv')
        
    del query_mess
    gc.collect()


In [ ]:
#Get list of peer ids
def from_influx_peerID(url, token, org, start_time, end_time):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "message" and r.node == "'+node+'" and r._field == "type" and (r._value == 11 or r._value == 12 or r._value == 3 or r._value == 2)) '
                                        ' |> group(columns: ["_tag"], mode: "by") '
                                        ' |> distinct(column: "peerID")')
                                        # ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')

    client.close()
    
    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_value']].rename(columns={"_value": "peerID"})#.sort_values(by=["_time"]).reset_index(drop=True)
    # df["_time"] = pd.to_datetime(df["_time"])

    return df

In [ ]:
execs = experiments#.drop(columns=['originalStart', 'originalEnd'])
# final=final.loc[final["identifier"] == 1]
peer = pd.DataFrame()

for idx, exec in execs.iterrows():
    query_mess = from_influx_peerID(url, token, org, exec["startUnix"], exec["endUnix"])

    peer = pd.concat([peer,query_mess])

peer=peer.reset_index()
peer=peer.drop_duplicates()
peerID = peer["peerID"].item()
print(peerID)
peer.head(100)

In [ ]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'" and r.peerID == "'+str(peerID)+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    if data_frame.empty == False:
        data_frame.reset_index(inplace=True)
        df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
        df["_time"] = pd.to_datetime(df["_time"])
    else:
        df = data_frame

    return df

In [ ]:
# gossip = pd.DataFrame()
execs = experiments#.drop(columns=['originalStart', 'originalEnd'])

for idx, exp in execs.iterrows():
    # print(exec["startUnix"], exec["endUnix"])
    query_iwant = from_influx(url, token, org, "iwant", exp["startUnix"], exp["endUnix"],"_measurement")

    if query_iwant.empty == False:
        query_iwant["experiment"] = exp["experiment"]
        query_iwant["_time"] = pd.to_datetime(query_iwant["_time"])

        query_iwant.to_csv('../../Datasets/partialTables/iwant_'+node+'_'+str(exp["experiment"])+'.csv')

        del query_iwant

    query_ihave = from_influx(url, token, org, "ihave", exp["startUnix"], exp["endUnix"],"_measurement")
    if query_ihave.empty == False:
        query_ihave["experiment"] = exp["experiment"]
        query_ihave["_time"] = pd.to_datetime(query_ihave["_time"])

        query_ihave.to_csv('../../Datasets/partialTables/ihave_'+node+'_'+str(exp["experiment"])+'.csv')
        
        del query_ihave

    gc.collect()

In [ ]:
execs = experiments#.drop(columns=['originalStart', 'originalEnd'])

for idx, exp in execs.iterrows():
    query_rpc = from_influx(url, token, org, "recvRPC", exp["startUnix"], exp["endUnix"],"_measurement")
    
    query_rpc["experiment"] = exp["experiment"]
    
    query_rpc["start"] = exp["start"]
    query_rpc["end"] = exp["end"]
            
    query_rpc["_time"] = pd.to_datetime(query_rpc["_time"])
    
    query_rpc.to_csv('../../Datasets/partialTables/rpc_'+node+'_'+str(exp["experiment"])+'.csv')
    
    del query_rpc
    gc.collect()
        

In [ ]:
for idx, exec in execs.iterrows():
    # print(exec["startUnix"], exec["endUnix"])
    query_mess = from_influx(url, token, org, "deliverMessage", exec["startUnix"], exec["endUnix"],"messageID")
    query_pub = from_influx(url, token, org, "publishMessage", exec["startUnix"], exec["endUnix"],"messageID")

    if query_mess.empty == False:
        query_mess["experiment"] = exec["experiment"]
        query_mess["_time"] = pd.to_datetime(query_mess["_time"])
        query_mess.to_csv('../../Datasets/partialTables/mess2_'+node+'_'+str(exec["experiment"])+'.csv')

    if query_pub.empty == False:
        query_pub["experiment"] = exec["experiment"]
        query_pub["_time"] = pd.to_datetime(query_pub["_time"])
        query_pub.to_csv('../../Datasets/partialTables/pub_'+node+'_'+str(exec["experiment"])+'.csv')

    del query_pub
    del query_mess

    gc.collect()